In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
# Load the Haar cascade for eye detection and your trained eye open/closed model
eye_cascade = cv2.CascadeClassifier(r'C:\Hope_Learning\Deep_learning\Supervised_learning\Eye_Open_or_Closed_Detection\Eye_detector\haarcascade_eye.xml')
model = load_model(r'C:\Hope_Learning\Deep_learning\Supervised_learning\Eye_Open_or_Closed_Detection\Eye_Detection_Model.h5')


In [3]:
# Start the video capture
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale (for eye detection)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect eyes in the frame
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in eyes:
        # Extract the eye region and resize it to the input size expected by your model
        eye = frame[y:y+h, x:x+w]
        eye = cv2.cvtColor(eye, cv2.COLOR_BGR2RGB)
        eye = cv2.resize(eye, (224, 224))  # Resize to match model input size
        eye = img_to_array(eye)
        eye = preprocess_input(eye)
        eye = np.expand_dims(eye, axis=0)  # Add batch dimension

        # Predict open or closed state
        prediction = model.predict(eye)
        ypredarg = np.argmax(prediction)

        # Determine the eye status based on prediction
        if ypredarg== 1:  # Adjust threshold as necessary
            status = "Open"
        else:
            status = "Closed"

        # Draw rectangle around the eye and display the status
        color = (0, 255, 0) if status == "Open" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, status, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Display the frame with results
    cv2.imshow('Eye Status Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 43ms/step
